<img src="http://www3.ifmg.edu.br/site_campi/p/images/logos/Betim-Simples.png" width="700"></img>
<b>Engenharia de Controle e Automação - Redes neurais artificiais</b> - Prof. Leandro Freitas

### Lucas Pevidor Reis - 0024855 - TP2

# 1  Objetivo

Implementar uma rede neural para estimar o valor de uma função com dois valores de entrada.


# 2 Regressão de uma função de duas variáveis

## 2.1 Função  a ser estimada:

A função a ser estimada é dada por $y = tan^{-1}(0,3x_1 + 0,7x_2)$. Inicialmente são gerados 20 valores de entrada de forma aleatória com distribuição uniforme entre -4 e 4, para $x_1$ e $x_2$;

In [81]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

x1 = np.transpose(np.array([np.random.uniform(-4,4,20)]))
x2 = np.transpose(np.array([np.random.uniform(-4,4,20)]))
x = np.append(x1, x2, axis=1)

O valor desejado é dado por $d = tan^{-1}(0.3x_1 + 0.7x_2) + \epsilon$, onde $\epsilon$ é um ruído aleatório, gaussiano de média nula, gerado para cada par $x_1$ e $x_2$

In [82]:
ruido = np.transpose(np.array([np.random.normal(scale=0.1, size=20)]))
d = np.arctan(0.3*x1 + 0.7*x2) + ruido

In [12]:
class RedeNeural:

    #Função de inicialização da classe
    def __init__(self, n_entradas, func_act, use_bias):
        #Se optar por utilizar bias, será adicionada uma coluna no final dos vetores de entrada com valor unitário.
        self.use_bias = use_bias        
        if use_bias:
            self.pesos = np.empty((n_entradas+1, 1), dtype=float)
        else:
            self.pesos = np.empty((n_entradas, 1), dtype=float)
        self.func_act = func_act   
    
    def reset(self):
        #Zerar pesos
        self.pesos = np.zeros((len(self.pesos[:,0]), 1), dtype=float)
            
    #Definição de entradas
    #Função utilizada para configurar o vetor de entrada interno da classe, juntamente com normalização, adição
    #da coluna de bias. Utilizar quando a média e desvpad para normalização ainda não foram definidos.
    def set_entradas_ant(self, entradas, normalizar):        
        if self.use_bias:
            vetor_1 = np.zeros([len(entradas[:,0]), 1]) + 1
            if normalizar:
                self.normalizar(entradas, False)
                self.vetor_entradas = np.append(self.entr_normalizada, vetor_1, axis=1)
            else:
                self.vetor_entradas = np.append(entradas, vetor_1, axis=1)
        else:
            if normalizar:                
                self.vetor_entradas = self.normalizar(entradas, False)
            else:
                self.vetor_entradas = entradas
    
    #Definir entradas após treinamento da rede
    #Função utilizada para configurar o vetor de entrada interno da classe, juntamente com normalização, adição
    #da coluna de bias. Utilizar quando a média e desvpad para normalização ainda JÁ foram definidos.
    def set_entradas_pós(self, entradas, normalizar):
        if self.use_bias:
            vetor_1 = np.zeros([len(entradas[:,0]), 1]) + 1
            if normalizar:            
                self.vetor_entradas = np.append(self.normalizar(entradas, True), vetor_1, axis=1)
            else:
                self.vetor_entradas = np.append(entradas, vetor_1, axis=1)
        else:
            if normalizar:
                self.vetor_entradas = self.normalizar(entradas, True)
            else:                    
                self.vetor_entradas = entradas
    
    def normalizar(self, entradas, utilizar_parametros_anteriores):
        if utilizar_parametros_anteriores == False:
            #Encontrar média e desvio padrão a partir das entradas fornecidas.
            #Executado antes do treinamento da rede
            self.media = np.empty(len(entradas[0,:]), dtype=float)
            self.desvpad = np.empty(len(entradas[0,:]), dtype=float)
            self.entr_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
        
            for i in range(0, len(entradas[0,:])):
                self.media[i] = np.mean(entradas[:,i])
                self.desvpad[i] = np.sqrt(np.var(entradas[:,i]))            
                self.entr_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]
            return self.entr_normalizada
        else:
            #Normaliza os dados de entrada fornecidos conforme a média e desvio padrão já encontrados anteriormente
            entr_nova_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
            for i in range(0, len(entradas[0,:])):                
                entr_nova_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]            
            
            return entr_nova_normalizada    
        
    def proc_saida(self, entradas, normalizado):
        if not normalizado:
            #Normalização das entradas utilizando média e desvpad já encontrados
            entr_nova_normalizada = self.normalizar(entradas, True)
        else:
            entr_nova_normalizada = entradas
        
        v = np.dot(entr_nova_normalizada, self.pesos)
        saida = np.empty([1,1])
        if(self.func_act == 'Bipolar'):
            
            for i in range(len(v)):
                if(v[i,:] > 0):
                    saida = np.append(saida, [[1]], axis=0)
                else:
                    saida = np.append(saida, [[-1]], axis=0)
            self.saida = saida[1:]
            return saida[1:]
        elif(self.func_act == 'Linear'):
            saida = v
            return saida
            
    def treinar_rede(self, fator_aprendizado, saida_desejada, max_iterações):
        redeTreinada = False
        self.historicoAcertos = []
        self.historicoPesos = np.empty([len(self.pesos), 1])
        
        for i in range(0, max_iterações):
            if not redeTreinada:
                flagAlteração = False
                acertos = 0
                for j in range(len(self.vetor_entradas[:,0])):
                    #Testa as entradas até encontrar um erro
                    #Caso encontrado, continua testando as outras entradas para obter a quantidade de acertos para
                    #os pesos atuais. Em seguida atualiza os pesos e reinicia os testes.
                    res = saida_desejada[j,0] - self.proc_saida([self.vetor_entradas[j,:]], True)[0,0]
                    if res == 0 and self.func_act=='Bipolar':
                        acertos += 1
                    else:
                        deltaW = fator_aprendizado * res * self.vetor_entradas[j,:]                    
                        flagAlteração = True                                                
                        
                        #Se a rede for bipolar, termina de checar a quantidade de acertos
                        #antes de atualizar os pesos
                        if self.func_act=='Bipolar':
                            for k in range(j+1, len(self.vetor_entradas[:,0])):
                                res = saida_desejada[k,0] - self.proc_saida([self.vetor_entradas[k,:]], True)[0,0]
                                if res == 0:
                                    acertos += 1
                        
                            #Salva os acertos e pesos
                            self.historicoAcertos.append(acertos)
                            self.historicoPesos = np.append(self.historicoPesos, self.pesos, axis=1)
                        
                        #Atualiza os pesos                               
                        self.pesos += np.transpose([deltaW])
                        break
                
                if flagAlteração == False:
                    redeTreinada = True
                    self.historicoAcertos.append(acertos)
                    self.historicoPesos = np.append(self.historicoPesos, self.pesos, axis=1)                
        
        self.historicoErros = []
        for i in range(len(self.historicoAcertos)):            
            self.historicoErros.append(len(self.vetor_entradas) - self.historicoAcertos[i])
        
        if redeTreinada == False:
            if self.func_act=='Bipolar':
                #Se não foi possível obter 100% de acerto durante as iterações
                #encontra e define como pesos os que obtiveram mais acertos
                indiceMax = self.historicoAcertos.index(max(self.historicoAcertos))                        
                self.pesos = np.transpose([self.historicoPesos[:, indiceMax+1]])
                print('Nº erros: {}'.format(len(self.vetor_entradas[:,0])-max(self.historicoAcertos)))
            

In [71]:
d = np.transpose([2*x[:,0] + 4*x[:,1]])
#d += ruido

linearNet = RedeNeural(2, 'Linear', False)
linearNet.set_entradas_ant(x, True)
linearNet.treinar_rede(0.1, d, 20)

In [72]:
linearNet.pesos = np.array([[2], [4]])

In [73]:
linearNet.set_entradas_pós(x, False)
print(linearNet.proc_saida(linearNet.vetor_entradas, True))
print(d)

[[  9.25012928]
 [  4.86321095]
 [-19.60890562]
 [ 11.59860392]
 [ -2.82356281]
 [ 16.89560705]
 [ -4.88346189]
 [ 13.47007325]
 [ -6.82441042]
 [  0.76588766]
 [ -2.12852902]
 [ -0.802849  ]
 [  6.64684158]
 [  3.64105613]
 [-13.43953012]
 [  8.69500435]
 [  0.68679049]
 [  5.49606076]
 [ -3.21605688]
 [ -0.20443243]]
[[  9.25012928]
 [  4.86321095]
 [-19.60890562]
 [ 11.59860392]
 [ -2.82356281]
 [ 16.89560705]
 [ -4.88346189]
 [ 13.47007325]
 [ -6.82441042]
 [  0.76588766]
 [ -2.12852902]
 [ -0.802849  ]
 [  6.64684158]
 [  3.64105613]
 [-13.43953012]
 [  8.69500435]
 [  0.68679049]
 [  5.49606076]
 [ -3.21605688]
 [ -0.20443243]]
